In [21]:
import os
import glob
import pandas as pd
from scipy import stats

In [30]:
os.chdir("/Data_Resource/m6A-seq/com_fetus_vs_adult/m6A_abundance/quantification/norm_by_scaleFactor")
result_dir = "/Data_Resource/m6A-seq/com_fetus_vs_adult/diff_methylation/ttest_result/norm_by_scaleFactor"
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

In [31]:
df = pd.read_table("bedtools_quantification.matrix")
df.head()

,brainadult1,brainadult2,brainadult3,brainadult4,brainadult5,brainadult6,brainfetus1,brainfetus2,brainfetus3,heartadult1,...,lungadult1,lungadult2,lungadult3,lungadult4,lungfetus1,lungfetus2,stomachadult1,stomachadult2,stomachfetus1,stomachfetus2
chr1:14421-14890,1.608945,2.525541,1.934167,1.469178,2.268553,2.166241,0.417069,0.353146,0.329119,1.724983,...,1.992934,1.836966,1.482626,1.795519,0.369157,0.283512,1.300315,1.871536,0.372790,0.360427
chr1:17663-17863,1.018309,0.976513,1.031975,1.121892,1.004199,1.205002,0.437480,0.477354,0.493127,1.022058,...,1.091146,1.000000,0.910606,1.197881,0.436171,0.609198,0.919379,1.000000,0.455636,0.558357
chr1:90808-91060,1.159836,1.472264,1.088819,1.021498,1.033504,1.702625,1.086813,1.072099,1.000000,1.052519,...,1.043403,1.000000,1.029319,1.000000,1.000000,1.000000,1.087398,1.410057,0.957760,1.000000
chr1:135063-135436,1.147629,1.713452,1.159621,1.312263,1.866669,1.112733,1.449358,1.050622,1.143532,1.603196,...,1.720335,1.509801,1.457210,1.424411,3.764722,3.465551,1.151833,0.692039,2.767643,2.272157
chr1:137651-137861,1.042476,2.029869,0.897951,0.954307,1.093237,1.006692,1.197510,1.139007,0.971698,1.357130,...,1.156251,0.935368,1.410953,1.000000,2.117016,0.708188,1.000000,0.550429,1.089790,1.063903


In [24]:
#equation for p-value correction (p-value x NumbTest / p-value_Rank)
#choose cutoff point for significance
def fdr_correction(p_values):
    adj_p_values=[]
    # get rank number of each pvalue
    tmp_list = np.argsort(np.argsort(p_values))
    rank_list = [x+1 for x in tmp_list]
    for i in range(0, len(p_values)):
        adj_p_values.append(p_values[i] * (len(p_values)) / rank_list[i])
    return adj_p_values

In [35]:
columns = df.columns
#for tissue in ["brain", "heart", "lung", "stomach"]: # "liver", 
for tissue in ["lung"]:
    print(tissue)
    selected_cols = [x for x in columns if tissue in x]
    adult_cols = [x for x in selected_cols if "adult" in x]
    fetus_cols = [x for x in selected_cols if "fetus" in x]
    df_tissue = df[selected_cols]
    m6a_list,pvalue_list, fc_list,value_list = [], [], [], []
    for m6a_id, values in df_tissue.iterrows():
        adult_values = [values[x] for x in adult_cols]
        fetus_values = [values[x] for x in fetus_cols]
        if (np.mean(adult_values)>1.5) or (np.mean(fetus_values)>1.5):
            m6a_list.append(m6a_id)
            result = stats.ttest_ind(fetus_values, adult_values)
            pvalue_list.append(result.pvalue)
            fc = np.median(adult_values) / np.median(fetus_values) ##### 
            fc_list.append(fc)
            str_v = "%s___%s" % (",".join([str(x) for x in adult_values]),",".join([str(x) for x in fetus_values]))
            value_list.append(str_v)
    qvalue_list = fdr_correction(pvalue_list)
    assert len(m6a_list) == len(pvalue_list) == len(qvalue_list) == len(value_list)
    with open("%s/%s_diff.txt" % (result_dir, tissue), 'w') as fw:
        for i in range(len(pvalue_list)):
            fw.write("%s\t%s\t%s\t%s\t%s\n" % (m6a_list[i], str(pvalue_list[i]), str(qvalue_list[i]), str(fc_list[i]), value_list[i]))

lung


#### post process

In [36]:
os.chdir(result_dir)
txt_list = glob.glob("*_diff.txt")
for x in txt_list:
    print(x)
    df = pd.read_table(x, header=None)
    # Both pvalue and qvalue < 0.05
    df_sub = df[(df.iloc[:,1]<0.05) & (df.iloc[:,2]<0.05) & ((df.iloc[:,3]<0.5)|(df.iloc[:,3]>2))]
    result = os.path.join("final_result", os.path.basename(x))
    df_sub.to_csv(result, sep="\t", index=False, header=False)
    
    
for x in ["final_result/brain_diff.txt", "final_result/heart_diff.txt", "final_result/lung_diff.txt"]:
    print(x)
    df = pd.read_table(x, sep="\t", header=None)
    df_up = df[df.iloc[:, 3]>1]
    df_down = df[df.iloc[:, 3]<=1]
    df_up.to_csv(x.replace("diff", "diff_up"), sep="\t", header=False, index=False)
    df_down.to_csv(x.replace("diff", "diff_down"), sep="\t", header=False, index=False)

stomach_diff.txt
lung_diff.txt
brain_diff.txt
liver_diff.txt
heart_diff.txt
final_result/brain_diff.txt
final_result/heart_diff.txt
final_result/lung_diff.txt
